In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from core.node import Tensor
from core.module import Linear,LogisticRegression,normalization

from core.module import MSELoss,BCELoss
from core.module import Linear,LogisticRegression
from core.opt import SGD,GCHECK
from core.node import Tensor
from core.functions import sigmoid
import torch 


In [2]:
path="../data/marks.txt"
def load_data(path):
    data=pd.read_csv(path,header=None,names=["x1","x2","y"])
    # data=data.loc[0:1]
    x=data[["x1","x2"]].to_numpy()

    y=data["y"].to_numpy()
    # x,y=x[0],y[0]
    x,y=x.reshape(-1,2),y.reshape(-1,1)
    
    x=(x-66)/358
    return x,y

path="../data/marks.txt"
x,y=load_data(path)

In [3]:
x_torch,y_torch=torch.tensor(x,dtype=float),torch.tensor(y)
x_torch,y_torch=x_torch.to(torch.float32),y_torch.to(torch.float32)

In [4]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=torch.nn.Linear(2,1)
    def forward(self,x):
        out1=self.fc(x)
        out2=torch.sigmoid(out1)
        return out2


In [5]:
model_torch=LR()
loss_torch=torch.nn.BCELoss()
out=model_torch(x_torch)
y_torch=y_torch.to(torch.float)

In [6]:
l_torch=loss_torch(out,y_torch)

l_torch.backward()
for n,p in model_torch.named_parameters():
    print(p.grad)

tensor([[-0.0150, -0.0129]])
tensor([-0.1145])


In [7]:
x,y=Tensor(x),Tensor(y)    

In [8]:
axis=0
ln=normalization(method="distrubution",axis=axis)
# x_n=ln(x)
x_n=x
model=LogisticRegression(2,1)
loss=BCELoss("mean")

output=model(x_n)
l=loss(output,y)
l.backward()


In [9]:
epslion=1e-5
optimizer=GCHECK(model.parameters(),epsilon=epslion)

its=optimizer.step()

grad_approx=np.array([])
for it in its:
    output1=model(x_n)
    l1=loss(output1,y)
    print(l1.data.item()-l.data.item())
    grad_one=-(l1.data.item()-l.data.item()) /epslion
    grad_approx=np.concatenate([grad_approx,np.array(grad_one).reshape(1)])

9.5367431640625e-07
1.1920928955078125e-07
1.1920928955078125e-07


array([-0.09536743, -0.01192093, -0.01192093])